In [1]:
import pandas as pd
import numpy as np

from math import floor, ceil

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
N_TOTAL_SEATS = 500

In [3]:
df = pd.read_csv('../inputs/2019-general-election.csv', index_col="party")
df

,n_votes,n_constituency_seats,is_party
party,,,
เพื่อไทย,7881006,136,1
พลังประชารัฐ,8441274,97,1
อนาคตใหม่,6330617,31,1
ประชาธิปัตย์,3959358,33,1
ภูมิใจไทย,3734459,39,1
เสรีรวมไทย,824284,0,1
ชาติไทยพัฒนา,783689,6,1
ประชาชาติ,481490,6,1
เศรษฐกิจใหม่,486273,0,1


In [4]:
N_CONSTITUENCY_SEATS = df["n_constituency_seats"].sum()
N_PROPORTION_SEATS = N_TOTAL_SEATS - N_CONSTITUENCY_SEATS

N_CONSTITUENCY_SEATS, N_PROPORTION_SEATS

(350, 150)

In [5]:
# มาตรา ๑๒๘ เมื่อคณะกรรมการได้รับรายงานผลรวมคะแนนแบบบัญชีรายชื่อจาก
# ผู้อำนวยการการเลือกตั้งประจำจังหวัดแล้ว ให้ดำเนินการคำนวณหาจำนวนสมาชิกสภาผู้แทนราษฎร
# ที่พรรคการเมืองจะพึงมีได้
# และให้ดำเนินการคำนวณเพื่อหาผู้ที่ได้รับเลือกตั้งเป็นสมาชิกสภาผู้แทนราษฎร
# แบบบัญชีรายชื่อ ดังต่อไปนี้โดยในกรณีที่มีเศษให้ใช้ทศนิยมสี่ตำแหน่ง

def use_four_decimal_place(n): return round(n, 4)
# def use_four_decimal_place(n): return n - (n % 0.0001)

In [6]:
# (๑) ให้รวมผลคะแนนทั้งหมดที่ทุกพรรคการเมืองได้รับจากการเลือกตั้งแบบบัญชีรายชื่อ
# ทั้งประเทศ

total_votes = df["n_votes"].sum()
total_votes

35080066

In [7]:
# (๒) ให้นำคะแนนรวมจาก (๑) หารด้วยหนึ่งห้าร้อยอันเป็นจำนวนสมาชิกทั้งหมด
# ของสภาผู้แทนราษฎร ผลลัพธ์ที่ได้ให้ถือเป็นคะแนนเฉลี่ยต่อสมาชิกสภาผู้แทนราษฎรแบบบัญชีรายชื่อ
# พึงมีหนึ่งคน
score_required_per_rep = total_votes / 500
score_required_per_rep

70160.132

In [8]:
# Filter non-party records

df = df[df["is_party"] == 1]
df

,n_votes,n_constituency_seats,is_party
party,,,
เพื่อไทย,7881006,136,1
พลังประชารัฐ,8441274,97,1
อนาคตใหม่,6330617,31,1
ประชาธิปัตย์,3959358,33,1
ภูมิใจไทย,3734459,39,1
เสรีรวมไทย,824284,0,1
ชาติไทยพัฒนา,783689,6,1
ประชาชาติ,481490,6,1
เศรษฐกิจใหม่,486273,0,1


In [9]:
# (๓) นำผลลัพธ์ตาม (๒) ไปหารจำนวน คะแนนรวมทั้งประเทศที่ได้รับจากการเลือกตั้ง
# แบบบัญชีรายชื่อของแต่ละพรรค ผลลัพธ์ที่ได้ให้ถือเป็นจำนวนสมาชิกสภาผู้แทนราษฎร
# ที่พรรคการเมืองนั้นจะพึงมีได้เบื้องต้น และเมื่อได้คำนวณตาม (๖) (๗) หรือ (๘) ถ้ามีแล้ว 
# จึงให้ถือว่าเป็นจำนวนสมาชิกสภาผู้แทนราษฎรที่พรรคการเมืองนั้นจะพึงมีได้

# `step_3_base` will be "จำนวนสมาชิกสภาผู้แทนราษฎรที่พรรคการเมืองนั้นจะพึงมีได้เบื้องต้น"

df['step_3'] = use_four_decimal_place(df["n_votes"] / score_required_per_rep)
df

,n_votes,n_constituency_seats,is_party,step_3
party,,,,
เพื่อไทย,7881006,136,1,112.3288
พลังประชารัฐ,8441274,97,1,120.3144
อนาคตใหม่,6330617,31,1,90.2310
ประชาธิปัตย์,3959358,33,1,56.4332
ภูมิใจไทย,3734459,39,1,53.2277
เสรีรวมไทย,824284,0,1,11.7486
ชาติไทยพัฒนา,783689,6,1,11.1700
ประชาชาติ,481490,6,1,6.8627
เศรษฐกิจใหม่,486273,0,1,6.9309


In [10]:
# (๔) นำจำนวนสมาชิกสภาผู้แทนราษฎรที่พรรคการเมือง
# จะพึงมีได้ตาม (๓) ลบด้วยจำนวนสมาชิกสภาผู้แทนราษฎรแบบแบ่งเขตเลือกตั้ง
# ทั้งหมดที่พรรคการเมืองนั้นได้รับเลือกตั้งในทุกเขตเลือกตั้ง
# ผลลัพธ์ที่ได้คือจำนวนสมาชิกสภาผู้แทนราษฎรแบบบัญชีรายชื่อพรรคการเมืองนั้นจะได้รับเบื้องต้น

df["step_4"] = df["step_3"] - df["n_constituency_seats"]
df

,n_votes,n_constituency_seats,is_party,step_3,step_4
party,,,,,
เพื่อไทย,7881006,136,1,112.3288,-23.6712
พลังประชารัฐ,8441274,97,1,120.3144,23.3144
อนาคตใหม่,6330617,31,1,90.2310,59.2310
ประชาธิปัตย์,3959358,33,1,56.4332,23.4332
ภูมิใจไทย,3734459,39,1,53.2277,14.2277
เสรีรวมไทย,824284,0,1,11.7486,11.7486
ชาติไทยพัฒนา,783689,6,1,11.1700,5.1700
ประชาชาติ,481490,6,1,6.8627,0.8627
เศรษฐกิจใหม่,486273,0,1,6.9309,6.9309


In [11]:
# (๖) ถ้าพรรคการเมืองใดมีผู้ได้รับเลือกตั้งเป็นสมาชิกสภาผู้แทนราษฎรแบบแบ่งเขตเลือกตั้ง
# เท่ากับหรือสูงกว่าจำนวนสมาชิกสภาผู้แทนราษฎรที่พรรคการเมืองนั้นจะพึงมีได้ตาม (๓)
# ให้พรรคการเมืองนั้นมีสมาชิกสภาผู้แทนราษฎรตามจำนวนที่ได้รับการเลือกตั้งแบบแบ่งเขตเลือกตั้ง 
# และไม่มีสิทธิได้รับการจัดสรรสมาชิกสภาผู้แทนราษฎรแบบบัญชีรายชื่อ และให้นำจำนวนสมาชิก
# สภาผู้แทนราษฎรแบบบัญชีรายชื่อทั้งหมดไปจัดสรรให้แก่พรรคการเมืองที่มีจำนวนสมาชิก
# สภาผู้แทนราษฎรแบบแบ่งเขตเลือกตั้งต่ำกว่าจำนวนสมาชิกสภาผู้แทนราษฎรที่พรรคการเมืองนั้นจะพึงมีได้
# ตาม (๓) ตามอัตราส่วน แต่ต้องไม่มีผลให้พรรคการเมืองใดดังกล่าวมีสมาชิกสภาผู้แทนราษฎร
# เกินจำนวนที่จะพึงมีได้ตาม (๓)

df["step_6_eligible_for_proportion"] = df["step_4"] > 0
df

,n_votes,n_constituency_seats,is_party,step_3,step_4,step_6_eligible_for_proportion
party,,,,,,
เพื่อไทย,7881006,136,1,112.3288,-23.6712,False
พลังประชารัฐ,8441274,97,1,120.3144,23.3144,True
อนาคตใหม่,6330617,31,1,90.2310,59.2310,True
ประชาธิปัตย์,3959358,33,1,56.4332,23.4332,True
ภูมิใจไทย,3734459,39,1,53.2277,14.2277,True
เสรีรวมไทย,824284,0,1,11.7486,11.7486,True
ชาติไทยพัฒนา,783689,6,1,11.1700,5.1700,True
ประชาชาติ,481490,6,1,6.8627,0.8627,True
เศรษฐกิจใหม่,486273,0,1,6.9309,6.9309,True


In [12]:
# (๗) ในกรณีที่คำนวณตาม (๖) แล้ว ปรากฏว่าพรรคการเมืองทุกพรรคได้รับจำนวน
# สมาชิกสภาผู้แทนราษฎรแบบบัญชีรายชื่อรวมกันแล้วเกินหนึ่งร้อยคน ให้ดำเนินการคำนวณปรับจำนวน
# สมาชิกสภาผู้แทนราษฎรแบบบัญชีรายชื่อใหม่ โดยคำนวณตามอัตราส่วนที่ทุกพรรคจะได้รับการจัดสรร
# จำนวนสมาชิกสภาผู้แทนราษฎรแบบบัญชีรายชื่อ ซึ่งเมื่อรวมแล้วไม่เกินหนึ่งร้อยคน โดยให้นำจำนวน
# สมาชิกสภาผู้แทนราษฎรแบบบัญชีรายชื่อที่แต่ละพรรคจะได้รับคูณด้วยหนึ่งร้อย หารด้วยผลบวกของ
# หนึ่งร้อยกับจำนวนสมาชิกสภาผู้แทนราษฎรแบบบัญชีรายชื่อที่เกินจำนวนหนึ่งร้อย และให้นำ (๕) มาใช้
# ในการคำนวณด้วยโดยอนุโลม

df_elig_proportion = df[df["step_6_eligible_for_proportion"]]
df_elig_proportion["step_5_scaled_porportion"] = df_elig_proportion["step_4"] / df_elig_proportion["step_4"].sum() * N_PROPORTION_SEATS
df_elig_proportion

,n_votes,n_constituency_seats,is_party,step_3,step_4,step_6_eligible_for_proportion,step_5_scaled_porportion
party,,,,,,,
พลังประชารัฐ,8441274,97,1,120.3144,23.3144,True,20.231884
อนาคตใหม่,6330617,31,1,90.2310,59.2310,True,51.399766
ประชาธิปัตย์,3959358,33,1,56.4332,23.4332,True,20.334977
ภูมิใจไทย,3734459,39,1,53.2277,14.2277,True,12.346583
เสรีรวมไทย,824284,0,1,11.7486,11.7486,True,10.195257
ชาติไทยพัฒนา,783689,6,1,11.1700,5.1700,True,4.486448
ประชาชาติ,481490,6,1,6.8627,0.8627,True,0.748638
เศรษฐกิจใหม่,486273,0,1,6.9309,6.9309,True,6.014530
เพื่อชาติ,421412,0,1,6.0064,6.0064,True,5.212263


In [13]:
# (๕) ภายใต้บังคับ (๖) ให้จัดสรรจำนวนสมาชิกสภาผู้แทนราษฎร
# แบบบัญชีรายชื่อที่พรรคการเมืองจะได้รับให้ครบหนึ่งร้อยคน โดยจัดสรรให้พรรคการเมืองตามผลลัพธ์
# ตาม (๔) เป็นจำนวนเต็มก่อน...

df_elig_proportion["step_5_full_seats"] = df_elig_proportion["step_5_scaled_porportion"].apply(floor)
df_elig_proportion["step_5_remainder"] = df_elig_proportion["step_5_scaled_porportion"].apply(lambda n: n % 1)
df_elig_proportion

,n_votes,n_constituency_seats,is_party,step_3,step_4,step_6_eligible_for_proportion,step_5_scaled_porportion,step_5_full_seats,step_5_remainder
party,,,,,,,,,
พลังประชารัฐ,8441274,97,1,120.3144,23.3144,True,20.231884,20,0.231884
อนาคตใหม่,6330617,31,1,90.2310,59.2310,True,51.399766,51,0.399766
ประชาธิปัตย์,3959358,33,1,56.4332,23.4332,True,20.334977,20,0.334977
ภูมิใจไทย,3734459,39,1,53.2277,14.2277,True,12.346583,12,0.346583
เสรีรวมไทย,824284,0,1,11.7486,11.7486,True,10.195257,10,0.195257
ชาติไทยพัฒนา,783689,6,1,11.1700,5.1700,True,4.486448,4,0.486448
ประชาชาติ,481490,6,1,6.8627,0.8627,True,0.748638,0,0.748638
เศรษฐกิจใหม่,486273,0,1,6.9309,6.9309,True,6.014530,6,0.014530
เพื่อชาติ,421412,0,1,6.0064,6.0064,True,5.212263,5,0.212263


In [14]:
# (cont from 5) ...หากยังไม่ครบหนึ่งร้อยคน 
# ให้พรรคการเมืองที่มีเศษจากการคำนวณมากที่สุดได้รับการจัดสรรจำนวน
# สมาชิกสภาผู้แทนราษฎรแบบบัญชีรายชื่อเพิ่มอีกหนึ่งคนตามลำดับจนครบจำนวนหนึ่งร้อยคน ในกรณีที่
# มีเศษเท่ากันให้ดำเนินการตาม (๗)

assigned_proportion_seats = df_elig_proportion["step_5_full_seats"].sum()
n_senators_to_assign_by_remainder = N_PROPORTION_SEATS - assigned_proportion_seats

n_senators_to_assign_by_remainder

13

In [15]:
parties_to_get_reps_from_remainder = df_elig_proportion.sort_values("step_5_remainder", ascending=False).head(n_senators_to_assign_by_remainder).index

parties_to_get_reps_from_remainder

Index(['พลังปวงชนไทย', 'พลังชาติไทย', 'ประชาภิวัฒน์', 'ประชาชาติ',
       'พลังไทยรักไทย', 'ไทยศรีวิไลย์', 'ครูไทยเพื่อประชาชน', 'ประชานิยม​',
       'รักษ์ผืนป่าประเทศไทย', 'พลังท้องถิ่นไท', 'ประชาธรรมไทย​',
       'ประชาชนปฏิรูป​', 'พลเมืองไทย​'],
      dtype='object', name='party')

In [16]:
df_elig_proportion["step_5_remainder_seats"] = 0
df_elig_proportion.loc[df_elig_proportion.index.isin(parties_to_get_reps_from_remainder), "step_5_remainder_seats"] = 1
df_elig_proportion

,n_votes,n_constituency_seats,is_party,step_3,step_4,step_6_eligible_for_proportion,step_5_scaled_porportion,step_5_full_seats,step_5_remainder,step_5_remainder_seats
party,,,,,,,,,,
พลังประชารัฐ,8441274,97,1,120.3144,23.3144,True,20.231884,20,0.231884,0
อนาคตใหม่,6330617,31,1,90.2310,59.2310,True,51.399766,51,0.399766,0
ประชาธิปัตย์,3959358,33,1,56.4332,23.4332,True,20.334977,20,0.334977,0
ภูมิใจไทย,3734459,39,1,53.2277,14.2277,True,12.346583,12,0.346583,0
เสรีรวมไทย,824284,0,1,11.7486,11.7486,True,10.195257,10,0.195257,0
ชาติไทยพัฒนา,783689,6,1,11.1700,5.1700,True,4.486448,4,0.486448,0
ประชาชาติ,481490,6,1,6.8627,0.8627,True,0.748638,0,0.748638,1
เศรษฐกิจใหม่,486273,0,1,6.9309,6.9309,True,6.014530,6,0.014530,0
เพื่อชาติ,421412,0,1,6.0064,6.0064,True,5.212263,5,0.212263,0


In [17]:
df_elig_proportion["step_5_proportional_seats"] = df_elig_proportion["step_5_full_seats"] + df_elig_proportion["step_5_remainder_seats"]

df_elig_proportion

,n_votes,n_constituency_seats,is_party,step_3,step_4,step_6_eligible_for_proportion,step_5_scaled_porportion,step_5_full_seats,step_5_remainder,step_5_remainder_seats,step_5_proportional_seats
party,,,,,,,,,,,
พลังประชารัฐ,8441274,97,1,120.3144,23.3144,True,20.231884,20,0.231884,0,20
อนาคตใหม่,6330617,31,1,90.2310,59.2310,True,51.399766,51,0.399766,0,51
ประชาธิปัตย์,3959358,33,1,56.4332,23.4332,True,20.334977,20,0.334977,0,20
ภูมิใจไทย,3734459,39,1,53.2277,14.2277,True,12.346583,12,0.346583,0,12
เสรีรวมไทย,824284,0,1,11.7486,11.7486,True,10.195257,10,0.195257,0,10
ชาติไทยพัฒนา,783689,6,1,11.1700,5.1700,True,4.486448,4,0.486448,0,4
ประชาชาติ,481490,6,1,6.8627,0.8627,True,0.748638,0,0.748638,1,1
เศรษฐกิจใหม่,486273,0,1,6.9309,6.9309,True,6.014530,6,0.014530,0,6
เพื่อชาติ,421412,0,1,6.0064,6.0064,True,5.212263,5,0.212263,0,5


In [18]:
df = df.join(df_elig_proportion.loc[:, ["step_5_proportional_seats"]]).fillna(0)
df["total_seats"] = df["n_constituency_seats"] + df["step_5_proportional_seats"]

In [19]:
df[["n_votes", "total_seats"]]

,n_votes,total_seats
party,,
เพื่อไทย,7881006,136.0
พลังประชารัฐ,8441274,117.0
อนาคตใหม่,6330617,82.0
ประชาธิปัตย์,3959358,53.0
ภูมิใจไทย,3734459,51.0
เสรีรวมไทย,824284,10.0
ชาติไทยพัฒนา,783689,10.0
ประชาชาติ,481490,7.0
เศรษฐกิจใหม่,486273,6.0
